In [89]:
import numpy as np
import pandas as pd
import math

# K MEAN

In [84]:
inf = 1e9
class K_mean():
    def __init__(self):
        # INITIALIZATION
        self.points = [
            (1,2),
            (3,3),
            (2,2),
            (8,8),
            (6,6),
            (7,7),
            (-3,-3),
            (-2,-4),
            (-7,-7)
        ]

        
    def dist(self, p1, p2):
        d = 0
        for i in range(len(p1)):
            d += (p1[i] - p2[i])**2
        return d**(0.5)

    
    def mean(self, points):
        x,y = [0,0]
        for p in points:
            x += p[0]
            y += p[1]
        x /= len(points)
        y /= len(points)
        return (x,y)

    
    def update(self, points):
        new_centroids = []
        for ps in points:
            new_centroids.append(self.mean(ps))

        return new_centroids
    
    
    def assign(self, centroids, points): 
        assigned_points = [ [] for i in range(len(centroids)) ]
        for p in points:
            min_dist = inf
            index = -1
            for i in range(len(centroids)):
                d = self.dist(centroids[i],p)
                if(d < min_dist):
                    index = i
                    min_dist = d
            assigned_points[index].append(p)
        return assigned_points
    
    
    def diff(self, c_old, c_new):
        ans = -inf
        for i in range(len(c_old)):
            d = abs(c_old[i][0] - c_new[i][0]) + abs(c_old[i][1] - c_new[i][1])
            ans = max(ans , d)
        return ans

    
    def inner_var(self, centroids, points):
        d = 0
        for i, centroid in enumerate(centroids):
            for point in points[i]:
                d += kmean.dist(point, centroid)
        return d
    
    
    def fit(self, starting_points):
        centroids = starting_points
        epsilon = 1e-6
        changes = 10
        while( changes > epsilon ):
            assigned_points = self.assign(centroids,self.points)
            new_centroids = self.update(assigned_points)
            changes = self.diff(centroids, new_centroids)
            centroids = new_centroids

        return centroids, self.assign(centroids, self.points)

In [86]:
kmean = K_mean()
starting_points = [(3,3), (2,2), (-3,3)]
starting_points_2 = [(-3,-3), (2,2), (-7,-7)]

### T4

In [87]:
centroids, points = kmean.fit(starting_points)
for i in range(len(centroids)):
    print ("centroids {}".format(centroids[i]))
    print ("\t points: {}".format(points[i]))
print ("score" ,kmean.inner_var(centroids, points) )

centroids (7.0, 7.0)
	 points: [(8, 8), (6, 6), (7, 7)]
centroids (2.0, 2.3333333333333335)
	 points: [(1, 2), (3, 3), (2, 2)]
centroids (-4.0, -4.666666666666667)
	 points: [(-3, -3), (-2, -4), (-7, -7)]
score 13.270123925348127


### T5.

In [88]:
centroids, points = kmean.fit(starting_points_2)
for i in range(len(centroids)):
    print ("centroids {}".format(centroids[i]))
    print ("\t points: {}".format(points[i]))
print ("score" ,kmean.inner_var(centroids, points) )

centroids (-2.5, -3.5)
	 points: [(-3, -3), (-2, -4)]
centroids (4.5, 4.666666666666667)
	 points: [(1, 2), (3, 3), (2, 2), (8, 8), (6, 6), (7, 7)]
centroids (-7.0, -7.0)
	 points: [(-7, -7)]
score 21.971875715522874


## T6.

Evaluate on summary of inside cluster distance of every cluster. (lower is better)

The first initialization had better score than the second one.

# Titanic

In [90]:
### READ DATA
train_url = "http://s3.amazonaws.com/assets.datacamp.com/course/Kaggle/train.csv"
train = pd.read_csv(train_url) #training set
test_url = "http://s3.amazonaws.com/assets.datacamp.com/course/Kaggle/test.csv"
test = pd.read_csv(test_url) #test set

In [91]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [92]:
train.tail()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.00,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.00,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.45,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.00,C148,C
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,NaN,Q
